In [1]:
from numba import jit
import math

@jit
def hypot(x, y):
    # Implementation from https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

In [2]:
hypot(3.0, 4.0)

5.0

In [3]:
hypot.py_func(3.0, 4.0)

5.0

In [4]:
%timeit hypot.py_func(3.0, 4.0)

446 ns ± 4.64 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
%timeit hypot(3.0, 4.0)

123 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [7]:
%timeit math.sqrt(3.0 ** 2 + 4.0 ** 2)

54.8 ns ± 0.951 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [8]:
%timeit math.hypot(3.0, 4.0)

86.9 ns ± 1.94 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [9]:
hypot.inspect_types()

hypot (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-1-2d77e48c1ef7>
# --- LINE 4 --- 

@jit

# --- LINE 5 --- 

def hypot(x, y):

    # --- LINE 6 --- 

    # Implementation from https://en.wikipedia.org/wiki/Hypot

    # --- LINE 7 --- 
    # label 0
    #   x = arg(0, name=x)  :: float64
    #   y = arg(1, name=y)  :: float64
    #   $2load_global.0 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $6call_function.2 = call $2load_global.0(x, func=$2load_global.0, args=[Var(x, <ipython-input-1-2d77e48c1ef7>:7)], kws=(), vararg=None)  :: (float64,) -> float64
    #   del x
    #   del $2load_global.0
    #   x.1 = $6call_function.2  :: float64
    #   del $6call_function.2

    x = abs(x);

    # --- LINE 8 --- 
    #   $10load_global.3 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $14call_function.5 = call $10load_global.3(y, func=$10lo

In [10]:
@jit
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

<ipython-input-10-9ab63214e97a>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "cannot_compile" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-10-9ab63214e97a> (3)

File "<ipython-input-10-9ab63214e97a>", line 3:
def cannot_compile(x):
    return x['key']
    ^

  @jit
/home/bitai/anaconda3/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "cannot_compile" was compiled in object mode without forceobj=True.

File "<ipython-input-10-9ab63214e97a>", line 2:
@jit
def cannot_compile(x):
^

  state.func_ir.loc))
/home/bitai/anaconda3/lib/python3.7/site-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#depre

'value'

In [11]:
@jit(nopython=True)
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1m[1] During: typing of argument at <ipython-input-11-42c374763781> (3)[0m
[1m
File "<ipython-input-11-42c374763781>", line 3:[0m
[1mdef cannot_compile(x):
[1m    return x['key']
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class 'dict'>[0m


In [12]:
@jit(nopython=True)
def ex1(x, y, out):
    for i in range(x.shape[0]):
        out[i] = x[i] + y[i]

In [13]:
import numpy as np

in1 = np.arange(10, dtype=np.float64)
in2 = 2 * in1 + 1
out = np.empty_like(in1)

print('in1:', in1)
print('in2:', in2)

ex1(in1, in2, out)

print('out:', out)

in1: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
in2: [ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.]
out: [ 1.  4.  7. 10. 13. 16. 19. 22. 25. 28.]


In [17]:
# This test will fail until you fix the ex1 function
np.testing.assert_almost_equal(out, np.hypot(in1, in2))

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 9 / 10 (90%)
Max absolute difference: 6.97620396
Max relative difference: 0.33182418
 x: array([ 1.,  4.,  7., 10., 13., 16., 19., 22., 25., 28.])
 y: array([ 1.       ,  3.1622777,  5.3851648,  7.6157731,  9.8488578,
       12.083046 , 14.3178211, 16.5529454, 18.7882942, 21.023796 ])

In [ ]:
# https://github.com/ContinuumIO/gtc2017-numba